In [ ]:
import json
import pandas as pd
from simple_salesforce import Salesforce, format_soql, format_external_id
from json import loads, dumps

In [ ]:
with open("../login.json", "r") as j:
    misDatos = json.load(j)

In [ ]:
sf = Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)

In [ ]:
brandGroupsIds = "Lipigas"
busquedaDatos = "FIND {" + brandGroupsIds + "} RETURNING Brand_Account__c(Id,Atributo_1_3__c,Valor_1_3__c,Nombre_Elemento_1__c,Tipo_Elemento_1__c,Brand_Groups__r.Empresa__c  )"
derty = dict(sf.search(busquedaDatos))

In [ ]:
derty

In [ ]:
brandGroupsIds = "a41RN0000000fNRYAY"



soql_result = sf.query(
    format_soql(
        "SELECT Id,Nombre_Elemento_1__c,Tipo_Elemento_1__c,Atributo_1_3__c,Valor_1_3__c   from Brand_Account__c WHERE Brand_Groups__c = {brandGroupId} and Tipo_de_Front__c = 'mobile' and Tipo_Elemento_1__c = 'C'",
        brandGroupId=brandGroupsIds,
    )
)
df = pd.DataFrame(soql_result.get("records"))
df = df.drop(columns=["attributes"])
# df = df.drop(columns="Id")
list_of_records = df.to_dict("index")






for listado in list_of_records:
    print(
        
         list_of_records[listado]["Id"]
        + " - "
        + list_of_records[listado]["Valor_1_3__c"]
        + " - "
        + list_of_records[listado]["Nombre_Elemento_1__c"]
    )


In [ ]:
list_of_records

In [ ]:
def traerId(rut):
    if len(rut) >= 8:
        busquedaDatos = "FIND {" + rut + "} RETURNING Account(Id,IdentificationId__pc)"
        derty = dict(sf.search(busquedaDatos))
        if derty["searchRecords"] != []:
            if derty["searchRecords"][0]["attributes"]["type"] == "Account":
                jote = derty["searchRecords"][0]["Id"]
                dertyd = derty["searchRecords"][0]["IdentificationId__pc"]
                print(jote + " " + dertyd)
                return jote
            else:
                print("no")
        else:
            print("vacio")
    else:
        return ""

In [ ]:
archivoBCI["Paciente__c"] = archivoBCI["IdentificationId__pc"].apply(
    lambda y: traerId(y)
)
archivoBCI["Programas_por_Empresas__c"] = "a1W8c000008TCWxEAO"
archivoBCI.head()

In [ ]:
desc = sf.Brand_Account__c.describe()
fields_name = [field['name'] for field in desc['fields']]
soql = "SELECT {} FROM Brand_Account__c WHERE Brand_Groups__c = 'a41RN0000000fNRYAY' ".format(','.join(fields_name))
results = sf.query_all(soql)


""" fields = [field.get('Brand_Account__c') for field in getattr(sf, Brand_Account__c).describe().get('fields')]

# Concat fields ready for inclusion in query string
fields = ',\n'.join(fields)

# Convert into SOQL query
soql = f"SELECT \n{fields} \nFROM {TABLE}" """
results

In [ ]:

def traerEmpresa(nombreEmpresa):
    vaBusquedaEmpresa = "FIND {"+ nombreEmpresa +"} IN Name Fields RETURNING Account(Id,Name where IsPersonAccount = false and Nueva_Estructura__c = true and Padre_Empresa__c = false  )"
    
    derty = dict(sf.search(vaBusquedaEmpresa))
    if (derty["searchRecords"] != []):
    #return derty["searchRecords"][0]["Id"]
        
        return derty["searchRecords"][0]["Name"]
        
    else:        
        return "0"
    
    return "0"

""" def traerPoliza(numeroPoliza):
    vaBusquedaEmpresa = "FIND {"+ numeroPoliza +"} RETURNING Datos_Empresa__c(Id,P_liza__c,Empresa__c, Identificador__c where Empresa__c = )"
    derty = dict(sf.search(vaBusquedaEmpresa))
    #if (derty["searchRecords"][0]["Poliza__c"] == nombreEmpresa):
    return derty["searchRecords"] 
    
asa = "Sherwin Williams Chile S.A."
poliza = "34001"


traerEmpresa(asa)
traerPoliza(poliza)
    
    """



In [ ]:
asa = "Sherwin Williams Chile S.A."
traerEmpresa(asa)


In [ ]:
nuevos = "F:/2024/empresa.xlsx"
empresasArchivo = pd.read_excel(nuevos)
empresasArchivo["empresa"] = empresasArchivo["empresa"].astype(str)

empresasArchivo["empresa"] = empresasArchivo["empresa"].str.title()
empresasArchivo["empresa"] = empresasArchivo["empresa"].str.replace("&", "And")
empresasArchivo["empresa"] = empresasArchivo["empresa"].str.replace("-", "And")


empresasArchivo.head(100)

In [ ]:
empresasArchivo.head(100)

In [ ]:
empresasArchivo["IdAccount"] = empresasArchivo["empresa"].apply(
    lambda y: traerEmpresa(y) 
)

In [ ]:
empresasArchivo.to_excel(
        "F:/2024/empresa1.xlsx", index=False
    )

In [ ]:
sf.Datos_Empresa__c.create({'Empresa__c':'001RN000004kd1IYAQ','Identificador__c':'example@example.com'})